In [2]:
import sys
sys.path.append('./VILA')


In [3]:
from llava.model import *


[2024-12-13 16:29:46,502] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [4]:
import os
import torch
from transformers import AutoModel, AutoTokenizer

model = "/media/m3/models/CholecT50_vila_13B_3_Epoch"
# find the model part that includes the the multimodal projector weights
model = AutoModel.from_pretrained(model, trust_remote_code=True, local_files_only=True)
checkpoint = model.state_dict()

for k, v in checkpoint.items():
    print(k, v.shape)


/media/m2/holoscan-dev/holoscan-ml/multi-modality-research/VILA/llava/model/llava_arch.py:108: UserWarning: model_dtype not found in config, defaulting to torch.float16.
  warnings.warn("model_dtype not found in config, defaulting to torch.float16.")
Loading checkpoint shards: 100%|██████████| 6/6 [00:14<00:00,  2.40s/it]


llm.model.embed_tokens.weight torch.Size([32000, 5120])
llm.model.layers.0.self_attn.q_proj.weight torch.Size([5120, 5120])
llm.model.layers.0.self_attn.k_proj.weight torch.Size([5120, 5120])
llm.model.layers.0.self_attn.v_proj.weight torch.Size([5120, 5120])
llm.model.layers.0.self_attn.o_proj.weight torch.Size([5120, 5120])
llm.model.layers.0.mlp.gate_proj.weight torch.Size([13824, 5120])
llm.model.layers.0.mlp.up_proj.weight torch.Size([13824, 5120])
llm.model.layers.0.mlp.down_proj.weight torch.Size([5120, 13824])
llm.model.layers.0.input_layernorm.weight torch.Size([5120])
llm.model.layers.0.post_attention_layernorm.weight torch.Size([5120])
llm.model.layers.1.self_attn.q_proj.weight torch.Size([5120, 5120])
llm.model.layers.1.self_attn.k_proj.weight torch.Size([5120, 5120])
llm.model.layers.1.self_attn.v_proj.weight torch.Size([5120, 5120])
llm.model.layers.1.self_attn.o_proj.weight torch.Size([5120, 5120])
llm.model.layers.1.mlp.gate_proj.weight torch.Size([13824, 5120])
llm.mod

In [14]:
import os
import torch
from transformers import AutoModel, AutoTokenizer

# find the model part that includes the the multimodal projector weights
model = AutoModel.from_pretrained('/media/m3/models/MiniCPM-V-2_6', trust_remote_code=True, ignore_mismatched_sizes=True)
checkpoint = model.state_dict()

for k, v in checkpoint.items():
    print(k)


/media/m3/conda/vila.cpp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:18<00:00,  4.51s/it]

llm.model.embed_tokens.weight
llm.model.layers.0.self_attn.q_proj.weight
llm.model.layers.0.self_attn.q_proj.bias
llm.model.layers.0.self_attn.k_proj.weight
llm.model.layers.0.self_attn.k_proj.bias
llm.model.layers.0.self_attn.v_proj.weight
llm.model.layers.0.self_attn.v_proj.bias
llm.model.layers.0.self_attn.o_proj.weight
llm.model.layers.0.mlp.gate_proj.weight
llm.model.layers.0.mlp.up_proj.weight
llm.model.layers.0.mlp.down_proj.weight
llm.model.layers.0.input_layernorm.weight
llm.model.layers.0.post_attention_layernorm.weight
llm.model.layers.1.self_attn.q_proj.weight
llm.model.layers.1.self_attn.q_proj.bias
llm.model.layers.1.self_attn.k_proj.weight
llm.model.layers.1.self_attn.k_proj.bias
llm.model.layers.1.self_attn.v_proj.weight
llm.model.layers.1.self_attn.v_proj.bias
llm.model.layers.1.self_attn.o_proj.weight
llm.model.layers.1.mlp.gate_proj.weight
llm.model.layers.1.mlp.up_proj.weight
llm.model.layers.1.mlp.down_proj.weight
llm.model.layers.1.input_layernorm.weight
llm.model

: 

In [ ]:
./build/bin/llama-minicpmv-cli -m /media/m3/models/MiniCPM-V-2_6/model/Model-7.6B-F16.gguf --mmproj /media/m3/models/MiniCPM-V-2_6/mmproj-model-f16.gguf --image tmp.jpg -c 4096 --temp 0.7 --top-p 0.8 --top-k 100 --repeat-penalty 1.05 -p "What is in the image?"

In [ ]:

# # get a list of mm tensor names
# mm_tensors = [k for k, v in checkpoint.items() if k.startswith("resampler")]

# # store these tensors in a new dictionary and torch.save them
# projector = {name: checkpoint[name].float() for name in mm_tensors}
# torch.save(projector, f"{args.model}/minicpmv.projector")

# clip_tensors = [k for k, v in checkpoint.items() if k.startswith("vpm")]
# if len(clip_tensors) > 0:
#     clip = {name.replace("vpm.", ""): checkpoint[name].float() for name in clip_tensors}
#     torch.save(clip, f"{args.model}/minicpmv.clip")

#     # added tokens should be removed to be able to convert Mistral models
#     if os.path.exists(f"{args.model}/added_tokens.json"):
#         with open(f"{args.model}/added_tokens.json", "w") as f:
#             f.write("{}\n")

# config = model.llm.config
# config.auto_map = {
#     "AutoConfig": "configuration_minicpm.MiniCPMConfig",
#     "AutoModel": "modeling_minicpm.MiniCPMModel",
#     "AutoModelForCausalLM": "modeling_minicpm.MiniCPMForCausalLM",
#     "AutoModelForSeq2SeqLM": "modeling_minicpm.MiniCPMForCausalLM",
#     "AutoModelForSequenceClassification": "modeling_minicpm.MiniCPMForSequenceClassification"
# }
# model.llm.save_pretrained(f"{args.model}/model")
# tok = AutoTokenizer.from_pretrained(args.model, trust_remote_code=True)
# tok.save_pretrained(f"{args.model}/model")

# print("Done!")
# print(f"Now you can convert {args.model} to a regular LLaMA GGUF file.")
# print(f"Also, use {args.model}/minicpmv.projector to prepare a minicpmv-encoder.gguf file.")

In [18]:
import gguf
from gguf import gguf_reader

# Load the GGUF file
model_path = "/media/m3/models/yi-vl/mmproj-model-f16-q6_k.gguf"
model = gguf_reader.GGUFReader(model_path)

for i in range(1000):
    try:
        print(model.get_tensor(i).name, model.get_tensor(i).shape)
    except Exception as e:
        break

mm.0.weight [1280 4096]
mm.0.bias [4096]
mm.1.weight [4096]
mm.1.bias [4096]
mm.3.weight [4096 4096]
mm.3.bias [4096]
mm.4.weight [4096]
mm.4.bias [4096]
v.class_embd [1280]
v.patch_embd.weight [  14   14    3 1280]
v.position_embd.weight [1280 1025]
v.pre_ln.weight [1280]
v.pre_ln.bias [1280]
v.blk.0.attn_k.weight [1280 1280]
v.blk.0.attn_k.bias [1280]
v.blk.0.attn_v.weight [1280 1280]
v.blk.0.attn_v.bias [1280]
v.blk.0.attn_q.weight [1280 1280]
v.blk.0.attn_q.bias [1280]
v.blk.0.attn_out.weight [1280 1280]
v.blk.0.attn_out.bias [1280]
v.blk.0.ln1.weight [1280]
v.blk.0.ln1.bias [1280]
v.blk.0.ffn_down.weight [1280 5120]
v.blk.0.ffn_down.bias [5120]
v.blk.0.ffn_up.weight [5120 1280]
v.blk.0.ffn_up.bias [1280]
v.blk.0.ln2.weight [1280]
v.blk.0.ln2.bias [1280]
v.blk.1.attn_k.weight [1280 1280]
v.blk.1.attn_k.bias [1280]
v.blk.1.attn_v.weight [1280 1280]
v.blk.1.attn_v.bias [1280]
v.blk.1.attn_q.weight [1280 1280]
v.blk.1.attn_q.bias [1280]
v.blk.1.attn_out.weight [1280 1280]
v.blk.1.attn

In [17]:
import gguf
from gguf import gguf_reader

# Load the GGUF file
model_path = "/media/m3/models/CholecT50_vila_13B_3_Epoch/mmproj-model-f16.gguf"
model = gguf_reader.GGUFReader(model_path)

for i in range(1000):
    try:
        print(model.get_tensor(i).name, model.get_tensor(i).shape)
    except Exception as e:
        break



mm.1.weight [4608]
mm.1.bias [4608]
mm.2.weight [4608 5120]
mm.2.bias [5120]
mm.4.weight [5120 5120]
mm.4.bias [5120]
v.patch_embd.weight [  14   14    3 1152]
v.patch_embd.bias [1152]
v.position_embd.weight [1152  729]
v.blk.0.attn_k.weight [1152 1152]
v.blk.0.attn_k.bias [1152]
v.blk.0.attn_v.weight [1152 1152]
v.blk.0.attn_v.bias [1152]
v.blk.0.attn_q.weight [1152 1152]
v.blk.0.attn_q.bias [1152]
v.blk.0.attn_out.weight [1152 1152]
v.blk.0.attn_out.bias [1152]
v.blk.0.ln1.weight [1152]
v.blk.0.ln1.bias [1152]
v.blk.0.ffn_down.weight [1152 4304]
v.blk.0.ffn_down.bias [4304]
v.blk.0.ffn_up.weight [4304 1152]
v.blk.0.ffn_up.bias [1152]
v.blk.0.ln2.weight [1152]
v.blk.0.ln2.bias [1152]
v.blk.1.attn_k.weight [1152 1152]
v.blk.1.attn_k.bias [1152]
v.blk.1.attn_v.weight [1152 1152]
v.blk.1.attn_v.bias [1152]
v.blk.1.attn_q.weight [1152 1152]
v.blk.1.attn_q.bias [1152]
v.blk.1.attn_out.weight [1152 1152]
v.blk.1.attn_out.bias [1152]
v.blk.1.ln1.weight [1152]
v.blk.1.ln1.bias [1152]
v.blk.1.

In [16]:
import gguf
from gguf import gguf_reader

# Load the GGUF file
model_path = "/media/m3/models/MiniCPM-V-2_6/mmproj-model-f16.gguf"
model = gguf_reader.GGUFReader(model_path)

for i in range(1000):
    try:
        print(model.get_tensor(i).name, model.get_tensor(i).shape)
    except Exception as e:
        break


resampler.query [3584   64]
resampler.pos_embed_k [3584 4900]
resampler.proj.weight [3584 3584]
resampler.kv.weight [1152 3584]
resampler.attn.q.weight [3584 3584]
resampler.attn.k.weight [3584 3584]
resampler.attn.v.weight [3584 3584]
resampler.attn.q.bias [3584]
resampler.attn.k.bias [3584]
resampler.attn.v.bias [3584]
resampler.attn.out.weight [3584 3584]
resampler.attn.out.bias [3584]
resampler.ln_q.weight [3584]
resampler.ln_q.bias [3584]
resampler.ln_kv.weight [3584]
resampler.ln_kv.bias [3584]
resampler.ln_post.weight [3584]
resampler.ln_post.bias [3584]
v.patch_embd.weight [  14   14    3 1152]
v.patch_embd.bias [1152]
v.position_embd.weight [1152 4900]
v.blk.0.attn_k.weight [1152 1152]
v.blk.0.attn_k.bias [1152]
v.blk.0.attn_v.weight [1152 1152]
v.blk.0.attn_v.bias [1152]
v.blk.0.attn_q.weight [1152 1152]
v.blk.0.attn_q.bias [1152]
v.blk.0.attn_out.weight [1152 1152]
v.blk.0.attn_out.bias [1152]
v.blk.0.ln1.weight [1152]
v.blk.0.ln1.bias [1152]
v.blk.0.ffn_down.weight [1152 43

In [ ]:
class DownSampleBlock(nn.Module):
    def forward(self, x):
        vit_embeds = x
        h = w = int(vit_embeds.shape[1] ** 0.5)
        vit_embeds = vit_embeds.reshape(vit_embeds.shape[0], h, w, -1)
        vit_embeds = self.flat_square(vit_embeds)
        vit_embeds = vit_embeds.reshape(vit_embeds.shape[0], -1, vit_embeds.shape[-1])
        return vit_embeds

    def flat_square(self, x):
        n, w, h, c = x.size()
        if w % 2 == 1:
            x = torch.concat([x, torch.zeros((n, 1, h, c), dtype=x.dtype).to(x.device)], dim=1).contiguous()
            n, w, h, c = x.size()
        if h % 2 == 1:
            x = torch.concat([x, torch.zeros((n, w, 1, c), dtype=x.dtype).to(x.device)], dim=2).contiguous()
            n, w, h, c = x.size()
        x = x.contiguous()
        # [batch, w, h/2, c*2]
        x = x.view(n, w, int(h / 2), int(c * 2))
        # [batch, h/2, w, c*2]
        x = x.permute(0, 2, 1, 3).contiguous()
        # [batch, h/2, w/2, c*4]
        x = x.view(n, int(h / 2), int(w / 2), int(c * 4))
        # [batch, w/2, h/2, c*4]
        x = x.permute(0, 2, 1, 3).contiguous()
        return x